In [12]:
import torch
from torch import nn
from tensorboardX import SummaryWriter
# from utils import save_checkpoint, use_optimizer
# from metrics import Metrics

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim_mlp']

        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = torch.nn.Linear(in_features=config['layer'][-1], out_features=1)
        self.logistic = torch.nn.Sigmoid()

        if config['weight_init_gaussian']:
            for sm in self.modules():
                if isinstance(sm, (nn.Embedding, nn.Linear)):
                    torch.nn.init.normal_(sm.weight.data, 0.0, 0.01)
        
    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        x = torch.cat([user_embedding, item_embedding], dim=-1)
        for idx, _ in enumerate(len(self.fc_layers)):
            x = self.fc_layers[idx](x)
            x = torch.nn.ReLU()(x)
        logits = self.affine_output(x)
        rating = self.logistic(logits)
        return rating


In [5]:
layers = [16]
for idx in enumerate(range(0)):
    print('hi')
    

In [ ]:
import pandas as pd
import math

class Metrics:
    def __init__(self, top_k):
        self._top_k = top_k
        self._subjects = None
    
    @property
    def subjects(self):
        return self._subjects
    
    @subjects.setter
    def subjects(self, subjects):
        test_users, test_items, test_scores ,neg_users, neg_items, neg_scores = \
            subjects[0], subjects[1], subjects[2],subjects[3], subjects[4], subjects[5]

        # the golden set
        test = pd.DataFrame({
            'user': test_users,
            'test_item': test_items,
            'test_score': test_scores
        })

        full = pd.DataFrame({
            'user': neg_users + test_users,
            'item': neg_items + test_items,
            'score': neg_scores + test_scores
        })

        full = pd.merge(full, test, on=['user'], how='left')

        full['rank'] = full.groupBy('user')['score'].rank(method='first', ascending=False)
        full.sort_values(['user', 'rank'], inplace=True)
        self._subjects = full
        
        
    def cal_hit_ratio(self):
        full, top_k = self._subjects, self._top_k
        top_k = full[full['rank'] <= top_k]
        test_in_top_k = top_k[top_k['test_item'] == top_k['item']]
        return len(test_in_top_k) * 1.0 / full['user'].nunique()
    
    def cal_ndcg(self):
        full, top_k = self._subjects, self._top_k
        top_k = full[full['rank'] <= top_k]
        test_in_top_k = top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x))
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()

In [ ]:
class Trainer:
    def __init__(self, model, config, generator, evaluator):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = model.to(self.device)
        self.config = config
        self.generator = generator
        self.evaluator = evaluator
        self.evaluate_data = self.generator.evaluate_data
        self.optimizer = self.get_optimizer()
        self.crit = torch.nn.BCELoss()

    def get_optimizer(self):
        if self.config['optimizer'] == 'sgd':
            optimizer = torch.optim.SGD(self.model.parameters(),
                                        lr=self.config['sgd']['lr'],
                                        momentum=self.config['sgd']['momentum'],
                                        weight_decay=self.config['sgd']['l2_regularization'])
        elif self.config['optimizer'] == 'adam':
            optimizer = torch.optim.Adam(self.model.parameters(),
                                        lr=self.config['adam']['lr'],
                                        weight_decay=self.config['adam']['l2_regularization'])
        elif self.config['optimizer'] == 'rmsprop':
            optimizer = torch.optim.RMSprop(self.model.parameters(),
                                            lr=self.config['rmsprop']['lr'],
                                            alpha=self.config['rmsprop']['alpha'],
                                            momentum=self.config['rmsprop']['momentum'])
        return optimizer

    def _train_batch(self, users, items, ratings):
        if self.config['use_cuda']:
            users, items, ratings = users.to(self.device), items.to(self.device), ratings.to(self.device)
        self.optimizer.zero_grad()
        ratings_pred = self.model(users, items)
        loss = self.crit(ratings_pred.view(-1), ratings)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        return loss

    def _train_epoch(self, train_loader, epoch_id):
        self.model.train()
        total_loss = 0
        for _, batch in enumerate(train_loader):
            user, item, rating = batch[0], batch[1], batch[2]
            rating = rating.float()
            loss = self._train_batch(user, item, rating)
            total_loss += loss
        return total_loss
    
    def evaluate_epoch(self):
        self.model.eval()
        with torch.no_grad():
            test_users, test_items, negative_users, negative_items = \
                  self.evaluate_data[0], self.evaluate_data[1], self.evaluate_data[2], self.evaluate_data[3]
            if self.config['use_cuda']:
                test_users, test_items, negative_users, negative_items = \
                    test_users.to(self.device), test_items.to(self.device), negative_users.to(self.device), negative_items.to(self.device)                                                                                                                            
            test_scores = self.model(test_users, test_items).cpu()
            negative_scores = self.model(negative_users, negative_items).cpu()
            test_users, test_items, negative_users, negative_items = \
                test_users.cpu(), test_items.cpu(), negative_users.cpu(), negative_items.cpu()
            self.evaluator.subjects = [
                test_users.detach().view(-1).tolist(),
                test_items.detach().view(-1).tolist(),
                test_scores.detach().view(-1).tolist(),
                negative_users.detach().view(-1).tolist(),
                negative_items.detach().view(-1).tolist(),
                negative_scores.detach().view(-1).tolist
            ]
        hit_ratio, ndcg = self.evaluator.cal_hit_ratio(), self.evaluator.cal_ndcg()
        return hit_ratio, ndcg
     
    def save(self, epoch_id, hit_ratio, ndcg):
        model_dir = self.config['model_dir'].format(self.config['alias'], epoch_id, hit_ratio, ndcg)
        torch.save(self.model.state_dict(), model_dir)
    
    def train(self):
        for epoch in range(self.config['num_epoch']):
            print('Epoch {} starts !'.format(epoch))
            print('-' * 80)
            train_loader = self.generator.get_train_loader(self.config['num_negative'], self.config['batch_size'])
            loss = self._train_epoch(train_loader, epoch_id=epoch)
            print('[Training Epoch {}] Batch {}, Loss {}'.format(epoch, loss))
            hit_ratio, ndcg = self.evaluate_epoch(epoch_id=epoch)
            self.save(epoch, hit_ratio, ndcg)



In [5]:
layer = [16, 64, 32, 16, 8]
layer[1:]

[64, 32, 16, 8]

In [10]:
for i, j in enumerate(zip(layer[:-1], layer[1:])):
    print(i)
    print(j)

0
(16, 64)
1
(64, 32)
2
(32, 16)
3
(16, 8)


In [2]:
mlp_config = {
            'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
            'num_epoch': 200,
            'batch_size': 256,  # 1024,
            'optimizer': {              
                'type': 'adam',
                'params': {
                    'lr' : 1e-3,
                    'l2_regularizarion': 0.0000001 # MLP model is sensitive to hyper params
                }
            },      
            'num_users': 6040,
            'num_items': 3706,
            'latent_dim': 8,
            'num_negative': 4,
            'layers': [16, 64, 32, 16, 8],  # layers[0] is the concat of latent user vector & latent item vector
            'weight_init_gaussian': True,
            'use_cuda': True,
            'device_id': 0,
            'pretrain': False,
            'pretrain_mf': 'checkpoints/{}'.format('gmf_factor8neg4_Epoch100_HR0.6391_NDCG0.2852.model'),
            'model_dir': 'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
            }


In [6]:
mlp_config['optimizer']['params']['lr']

0.001